In [ ]:
pip install -r requirements.txt

     |████████████████████████████████| 81kB 3.3MB/s 
     |████████████████████████████████| 66.2MB 86kB/s 
     |████████████████████████████████| 256kB 46.9MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 11.6MB 43.2MB/s 
     |████████████████████████████████| 163kB 46.2MB/s 
     |████████████████████████████████| 81kB 10.1MB/s 
     |████████████████████████████████| 81kB 10.1MB/s 
     |████████████████████████████████| 112kB 51.5MB/s 
     |████████████████████████████████| 143kB 45.5MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
  Created wheel for optuna: filename=optuna-2.2.0-cp36-none-any.whl size=340928 sha256=092c15af3d1583b6a6e6a19af13bb094a5e16c6571e81befa930583f79ced5af
  Stored in directory: /root/.cache/pip/wheels/a6/27/26/3ef57a8fb2c8bceef3f4ece73483cec1895e06eae442d34a41
Successfully built optuna
  Created wheel for PrettyTable: fi

# Data Importing

In [ ]:
import pandas as pd
import numpy as np
import category_encoders as ce
import miceforest as mf
import optuna
import lightgbm as lgb
import xgboost as xgb

from utils import *
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso
from sklearn import metrics
from sklearn.impute import SimpleImputer

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df = pd.read_csv('after_prep.csv')
df.head()

,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Seats,Price,Brand,Series,Type,Mileage (kmpl),Engine (CC),Power (bhp)
0,Mumbai,2010,72000,CNG,Manual,First,5.0,1.75,Maruti,Wagon,R,26.60,998.0,58.16
1,Pune,2015,41000,Diesel,Manual,First,5.0,12.50,Hyundai,Creta,1.6,19.67,1582.0,126.20
2,Chennai,2011,46000,Petrol,Manual,First,5.0,4.50,Honda,Jazz,V,18.20,1199.0,88.70
3,Chennai,2012,87000,Diesel,Manual,First,7.0,6.00,Maruti,Ertiga,VDI,20.77,1248.0,88.76
4,Coimbatore,2013,40670,Diesel,Automatic,Second,5.0,17.74,Audi,A4,New,15.20,1968.0,140.80


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6019 entries, 0 to 6018
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Location           6019 non-null   object 
 1   Year               6019 non-null   int64  
 2   Kilometers_Driven  6019 non-null   int64  
 3   Fuel_Type          6019 non-null   object 
 4   Transmission       6019 non-null   object 
 5   Owner_Type         6019 non-null   object 
 6   Seats              5976 non-null   float64
 7   Price              6019 non-null   float64
 8   Brand              6019 non-null   object 
 9   Series             6019 non-null   object 
 10  Type               6019 non-null   object 
 11  Mileage (kmpl)     5951 non-null   float64
 12  Engine (CC)        5983 non-null   float64
 13  Power (bhp)        5876 non-null   float64
dtypes: float64(5), int64(2), object(7)
memory usage: 658.5+ KB


# Preprocessing

In [ ]:
# Delete outlier
df = df[~(df.Kilometers_Driven > 1e6)]
df.shape

(6018, 14)

## Feature enginering

In [ ]:
# from itertools import combinations
# cat_cols = ['Location', 'Fuel_Type', 'Transmission', 'Owner_Type', 'Brand']

# for col in combinations(cat_cols, 2):
#     df[col[0]+'_'+col[1]] = df[col[0]] + "_" + df[col[1]]
    
# df.head()

In [ ]:
# Make categorical feature interactions
from sklearn.preprocessing import LabelEncoder
from itertools import combinations

object_cols = df.select_dtypes("object").columns
low_cardinality_cols = [col for col in object_cols if df[col].nunique() < 15]
low_cardinality_cols.append('Brand')
interactions = pd.DataFrame(index=df.index)

for features in combinations(low_cardinality_cols,2):
    
    new_interaction = df[features[0]].map(str)+"_"+df[features[1]].map(str)
    
    # encoder = LabelEncoder()
    interactions["_".join(features)] = new_interaction

df = df.join(interactions)
df.head()

,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Seats,Price,Brand,Series,Type,Mileage (kmpl),Engine (CC),Power (bhp),Location_Fuel_Type,Location_Transmission,Location_Owner_Type,Location_Brand,Fuel_Type_Transmission,Fuel_Type_Owner_Type,Fuel_Type_Brand,Transmission_Owner_Type,Transmission_Brand,Owner_Type_Brand
0,Mumbai,2010,72000,CNG,Manual,First,5.0,1.75,Maruti,Wagon,R,26.60,998.0,58.16,Mumbai_CNG,Mumbai_Manual,Mumbai_First,Mumbai_Maruti,CNG_Manual,CNG_First,CNG_Maruti,Manual_First,Manual_Maruti,First_Maruti
1,Pune,2015,41000,Diesel,Manual,First,5.0,12.50,Hyundai,Creta,1.6,19.67,1582.0,126.20,Pune_Diesel,Pune_Manual,Pune_First,Pune_Hyundai,Diesel_Manual,Diesel_First,Diesel_Hyundai,Manual_First,Manual_Hyundai,First_Hyundai
2,Chennai,2011,46000,Petrol,Manual,First,5.0,4.50,Honda,Jazz,V,18.20,1199.0,88.70,Chennai_Petrol,Chennai_Manual,Chennai_First,Chennai_Honda,Petrol_Manual,Petrol_First,Petrol_Honda,Manual_First,Manual_Honda,First_Honda
3,Chennai,2012,87000,Diesel,Manual,First,7.0,6.00,Maruti,Ertiga,VDI,20.77,1248.0,88.76,Chennai_Diesel,Chennai_Manual,Chennai_First,Chennai_Maruti,Diesel_Manual,Diesel_First,Diesel_Maruti,Manual_First,Manual_Maruti,First_Maruti
4,Coimbatore,2013,40670,Diesel,Automatic,Second,5.0,17.74,Audi,A4,New,15.20,1968.0,140.80,Coimbatore_Diesel,Coimbatore_Automatic,Coimbatore_Second,Coimbatore_Audi,Diesel_Automatic,Diesel_Second,Diesel_Audi,Automatic_Second,Automatic_Audi,Second_Audi


## Train test split

In [ ]:
# melakukan train test split di awal auntuk mencegah data bocor ke test set saat dilakukan encoding/imputation
features = df.drop(columns=['Price'])
target = df['Price']
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.25, random_state=0)

## Encoding

In [ ]:
# # Define category mapping for label encoding
# owner_map = {
#     'First': 1, 
#     'Second': 2, 
#     'Third': 3, 
#     'Fourth & Above': 4
# }
# # Encoding train set
# X_train["Owner_Type"] = X_train["Owner_Type"].map(owner_map)
# # Encoding test set
# X_test["Owner_Type"] = X_test["Owner_Type"].map(owner_map)

In [ ]:
# One hot encoding
col_to_encode = ['Location', 'Fuel_Type', 'Transmission', 'Owner_Type', 'Brand']
oh_encoder = ce.OneHotEncoder(cols=col_to_encode,
                              use_cat_names=True)
oh_encoder.fit(X_train)

# Encoding train set
X_train = oh_encoder.transform(X_train)
# Encoding test set
X_test = oh_encoder.transform(X_test)

/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning:

is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead



In [ ]:
# Target encoding
col_to_encode = X_train.select_dtypes("object").columns
encoder = ce.TargetEncoder(cols=col_to_encode)
encoder.fit(X_train, y_train)

# Encoding train set
X_train = encoder.transform(X_train)
# Encoding test set
X_test = encoder.transform(X_test)

/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning:

is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead



## Missing Value Imputation

In [ ]:
# memprediksi nilai missing value dengan algoritma 
imputer = mf.KernelDataSet(
  X_train,
  save_all_iterations=True,
  random_state=1991,
  mean_match_candidates=5
)
imputer.mice(10)

In [ ]:
# Train set imputation
X_train_full = imputer.complete_data()

In [ ]:
# Test set imputation
new_data = imputer.impute_new_data(X_test)
X_test_full = new_data.complete_data()

# Modeling

## Functions

In [ ]:
def get_cv_score(models, X_train, y_train):
    
    cv = KFold(n_splits=5, shuffle=True, random_state=0)
    summary = []
    for label, model in models.items():
        cv_results = cross_validate(model, X_train, y_train, cv=cv, 
                                    scoring=['r2',
                                             'neg_root_mean_squared_error',
                                             'neg_mean_absolute_error'])
        
        temp = pd.DataFrame(cv_results).copy()
        temp['Model'] = label
        summary.append(temp)
    
    summary = pd.concat(summary)
    summary = summary.groupby('Model').mean()
    
    summary.drop(columns=['fit_time', 'score_time'], inplace=True)
    summary.columns = ['CV R2', 'CV RMSE', 'CV MAE']
    summary[['CV RMSE', 'CV MAE']] = summary[['CV RMSE', 'CV MAE']] * -1
    
    return summary

In [ ]:
def evaluate_model(models, X_train, X_test, y_train, y_test):

    summary = {'Model':[], 'Train R2':[], 'Train RMSE':[], 'Train MAE':[],
               'Test R2':[], 'Test RMSE':[], 'Test MAE':[]}

    for label, model in models.items():
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        summary['Model'].append(label)

        summary['Train R2'].append(
            metrics.r2_score(y_train, y_train_pred))
        summary['Train RMSE'].append(
            np.sqrt(metrics.mean_squared_error(y_train, y_train_pred)))
        summary['Train MAE'].append(
            metrics.mean_absolute_error(y_train, y_train_pred))

        summary['Test R2'].append(
            metrics.r2_score(y_test, y_test_pred))
        summary['Test RMSE'].append(
            np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))
        summary['Test MAE'].append(
            metrics.mean_absolute_error(y_test, y_test_pred))
    
    summary = pd.DataFrame(summary)
    summary.set_index('Model', inplace=True)

    cv_scores = get_cv_score(models, X_train, y_train)
    summary = summary.join(cv_scores)
    summary.reset_index(inplace=True)
    summary = summary[['Train R2', 'CV R2', 'Test R2',
                       'Train RMSE', 'CV RMSE', 'Test RMSE',
                       'Train MAE', 'CV MAE', 'Test MAE', 'Model']]
    
    return round(summary.sort_values(by='CV RMSE'), 4)

## Hyperparameter Tuning

In [ ]:
X_train_full.head()

,Location_Mumbai,Location_Jaipur,Location_Bangalore,Location_Coimbatore,Location_Kochi,Location_Hyderabad,Location_Chennai,Location_Delhi,Location_Ahmedabad,Location_Kolkata,Location_Pune,Year,Kilometers_Driven,Fuel_Type_Petrol,Fuel_Type_Diesel,Fuel_Type_CNG,Fuel_Type_LPG,Fuel_Type_Electric,Transmission_Manual,Transmission_Automatic,Owner_Type_Third,Owner_Type_First,Owner_Type_Second,Owner_Type_Fourth & Above,Seats,Brand_Hyundai,Brand_Honda,Brand_Maruti,Brand_Ford,Brand_Mercedes-Benz,Brand_Audi,Brand_Land,Brand_Volkswagen,Brand_Toyota,Brand_Chevrolet,Brand_BMW,Brand_Renault,Brand_Mahindra,Brand_Tata,Brand_Jaguar,Brand_Skoda,Brand_Jeep,Brand_Nissan,Brand_Fiat,Brand_Mitsubishi,Brand_Datsun,Brand_Mini,Brand_Porsche,Brand_Volvo,Brand_Isuzu,Brand_Lamborghini,Brand_Bentley,Brand_Smart,Brand_Ambassador,Brand_Force,Series,Type,Mileage (kmpl),Engine (CC),Power (bhp),Location_Fuel_Type,Location_Transmission,Location_Owner_Type,Location_Brand,Fuel_Type_Transmission,Fuel_Type_Owner_Type,Fuel_Type_Brand,Transmission_Owner_Type,Transmission_Brand,Owner_Type_Brand
245,1,0,0,0,0,0,0,0,0,0,0,2007,72000,1,0,0,0,0,1,0,1,0,0,0,5.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.791967,1.738571,17.00,1086.0,68.0,6.317074,5.357432,5.064541,5.433222,4.125745,5.319811,4.579892,2.764203,4.966540,2.875729
2175,0,1,0,0,0,0,0,0,0,0,0,2013,70961,0,1,0,0,0,1,0,0,1,0,0,5.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.766282,3.978158,25.80,1498.0,98.6,7.980581,4.624060,6.232081,4.705517,6.607072,13.165977,6.063297,5.644766,5.151638,5.801518
4134,0,0,1,0,0,0,0,0,0,0,0,2016,34004,1,0,0,0,0,1,0,0,1,0,0,7.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8.579124,9.060769,15.40,1497.0,117.3,6.983143,6.301750,14.677045,6.022857,4.125745,6.085327,5.346462,5.644766,5.151638,5.801518
4975,0,0,0,1,0,0,0,0,0,0,0,2016,41810,1,0,0,0,0,1,0,0,1,0,0,5.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7.626034,6.809325,20.73,1373.0,91.1,7.765213,7.471327,15.376571,6.143857,4.125745,6.085327,3.670976,5.644766,4.418040,4.775952
2473,0,0,0,1,0,0,0,0,0,0,0,2018,68737,0,1,0,0,0,1,0,0,1,0,0,5.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7.626034,6.349655,26.21,1248.0,88.5,21.253077,7.471327,15.376571,6.143857,6.607072,13.165977,5.884225,5.644766,4.418040,4.775952


### XGBoost

#### Study 1

In [ ]:
 def objective(trial):

    dtrain = xgb.DMatrix(X_train_full, label=y_train)

    param = {
        'objective': 'reg:squarederror',
        'tree_method': 'hist',
        'learning_rate': 0.1,
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 6),
        'gamma': trial.suggest_loguniform("gamma", 1e-8, 1.0),
        'subsample':trial.suggest_uniform('subsample', 0.1, 1),
        'colsample_bytree':trial.suggest_uniform('colsample_bytree', 0.1, 1),
        "lambda": trial.suggest_loguniform("lambda", 1e-8, 1.0),
        "alpha": trial.suggest_loguniform("alpha", 1e-8, 1.0),
        "grow_policy": trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    }

    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "test-rmse")

    cv = KFold(n_splits=5, shuffle=True, random_state=0)
    history = xgb.cv(param, dtrain, num_boost_round=2000, 
                     early_stopping_rounds=100,
                     callbacks=[pruning_callback],
                     metrics='rmse', 
                     folds=cv)

    mean_score = history["test-rmse-mean"].values[-1]
    return mean_score

pruner = optuna.pruners.MedianPruner(n_warmup_steps=10)
study = optuna.create_study(pruner=pruner, direction='minimize')
study.optimize(objective, n_trials=1000)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2020-10-29 17:07:58,984] A new study created in memory with name: no-name-be5ce266-f0cd-41df-9426-2e22519f4de9
[I 2020-10-29 17:08:07,886] Trial 0 finished with value: 4.354630200000001 and parameters: {'max_depth': 1, 'min_child_weight': 4, 'gamma': 1.9182253261263595e-05, 'subsample': 0.14464959440630654, 'colsample_bytree': 0.8761114580970323, 'lambda': 3.616578619493552e-07, 'alpha': 1.954312629982336e-07, 'grow_policy': 'lossguide'}. Best is trial 0 with value: 4.354630200000001.
[I 2020-10-29 17:08:33,037] Trial 1 finished with value: 3.3323376000000002 and parameters: {'max_depth': 9, 'min_child_weight': 5, 'gamma': 0.001050434383778008, 'subsample': 0.7787862530969609, 'colsample_bytree': 0.11111196324586413, 'lambda': 0.00014173578761003645, 'alpha': 0.06158566988479406, 'grow_policy': 'depthwise'}. Best is trial 1 with value: 3.3323376000000002.
[I 2020-10-29 17:08:39,895] Trial 2 finished with value: 3.11187 and parameters: {'max_depth': 10, 'min_child_weight': 6, 'gamma'

Number of finished trials: 1000
Best trial:
  Value: 2.8860889999999997
  Params: 
    max_depth: 9
    min_child_weight: 2
    gamma: 1.9014577086891506e-05
    subsample: 0.6858450695750884
    colsample_bytree: 0.7471514347883379
    lambda: 3.327806502673549e-07
    alpha: 4.2651831704024965e-06
    grow_policy: depthwise


In [ ]:
# Get best params then add to param_1
study_1_params = study.best_params
param_1 = {
    'objective': 'reg:squarederror',
    'tree_method': 'hist',
    'learning_rate': 0.1,
}
param_1.update(study_1_params)
param_1

{'alpha': 4.2651831704024965e-06,
 'colsample_bytree': 0.7471514347883379,
 'gamma': 1.9014577086891506e-05,
 'grow_policy': 'depthwise',
 'lambda': 3.327806502673549e-07,
 'learning_rate': 0.1,
 'max_depth': 9,
 'min_child_weight': 2,
 'objective': 'reg:squarederror',
 'subsample': 0.6858450695750884,
 'tree_method': 'hist'}

In [ ]:
dtrain = xgb.DMatrix(X_train_full, label=y_train)

cv = KFold(
    n_splits=5, 
    shuffle=True, 
    random_state=0
)
history = xgb.cv(
    param_1, dtrain, 
    num_boost_round=2000, 
    early_stopping_rounds=100,
    metrics='rmse',
    folds=cv
)
n_estimators_1 = history.shape[0]
n_estimators_1

129

#### Study 2

In [ ]:
def objective(trial):

    dtrain = xgb.DMatrix(X_train_full, label=y_train)

    param = {
        'objective': 'reg:squarederror',
        'tree_method': 'hist',
        'learning_rate': 0.01,
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 6),
        'gamma': trial.suggest_loguniform("gamma", 1e-8, 1.0),
        'subsample':trial.suggest_uniform('subsample', 0.1, 1),
        'colsample_bytree':trial.suggest_uniform('colsample_bytree', 0.1, 1),
        "lambda": trial.suggest_loguniform("lambda", 1e-8, 1.0),
        "alpha": trial.suggest_loguniform("alpha", 1e-8, 1.0),
        "grow_policy": trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    }

    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "test-rmse")

    cv = KFold(n_splits=5, shuffle=True, random_state=0)
    history = xgb.cv(param, dtrain, num_boost_round=2000, 
                     early_stopping_rounds=100,
                     callbacks=[pruning_callback],
                     metrics='rmse', 
                     folds=cv)

    mean_score = history["test-rmse-mean"].values[-1]
    return mean_score

pruner = optuna.pruners.MedianPruner(n_warmup_steps=10)
study = optuna.create_study(pruner=pruner, direction='minimize')
study.optimize(objective, n_trials=1000)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2020-10-29 17:57:53,996] A new study created in memory with name: no-name-345c77db-f0f3-412f-8423-8783eca46b49
[I 2020-10-29 17:59:00,852] Trial 0 finished with value: 3.0585982 and parameters: {'max_depth': 9, 'min_child_weight': 3, 'gamma': 4.5830405167378046e-07, 'subsample': 0.5935191905943684, 'colsample_bytree': 0.3559133952432967, 'lambda': 0.16464229130909888, 'alpha': 0.014073811441187856, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 3.0585982.
[I 2020-10-29 17:59:34,225] Trial 1 finished with value: 3.0649583999999996 and parameters: {'max_depth': 5, 'min_child_weight': 3, 'gamma': 9.483081150337862e-06, 'subsample': 0.24924477796063224, 'colsample_bytree': 0.355177387974956, 'lambda': 1.1840602393107403e-06, 'alpha': 1.6029506034532946e-05, 'grow_policy': 'lossguide'}. Best is trial 0 with value: 3.0585982.
[I 2020-10-29 18:00:06,213] Trial 2 finished with value: 2.9964330000000006 and parameters: {'max_depth': 9, 'min_child_weight': 2, 'gamma': 1.572445066363

Number of finished trials: 1000
Best trial:
  Value: 2.9165078
  Params: 
    max_depth: 8
    min_child_weight: 2
    gamma: 0.08064916113283055
    subsample: 0.6417174665962085
    colsample_bytree: 0.7356482505189391
    lambda: 4.93323633930299e-08
    alpha: 0.00023526849350579993
    grow_policy: lossguide


In [ ]:
# Get best params then add to param_2
study_2_params = study.best_params
param_2 = {
    'objective': 'reg:squarederror',
    'tree_method': 'hist',
    'learning_rate': 0.01,
}
param_2.update(study_2_params)
param_2

{'alpha': 0.00023526849350579993,
 'colsample_bytree': 0.7356482505189391,
 'gamma': 0.08064916113283055,
 'grow_policy': 'lossguide',
 'lambda': 4.93323633930299e-08,
 'learning_rate': 0.01,
 'max_depth': 8,
 'min_child_weight': 2,
 'objective': 'reg:squarederror',
 'subsample': 0.6417174665962085,
 'tree_method': 'hist'}

In [ ]:
dtrain = xgb.DMatrix(X_train_full, label=y_train)

cv = KFold(
    n_splits=5, 
    shuffle=True, 
    random_state=0
)
history = xgb.cv(
    param_2, dtrain, 
    num_boost_round=2000, 
    early_stopping_rounds=100,
    metrics='rmse',
    folds=cv
)
n_estimators_2 = history.shape[0]
n_estimators_2

1778

#### Evaluation

In [ ]:
xgb_study_1 = XGBRegressor(**param_1, n_estimators=n_estimators_1)
xgb_study_2 = XGBRegressor(**param_2, n_estimators=n_estimators_2)

models = {
    f'XGBRegressor ({n_estimators_1}) {param_1}': xgb_study_1,
    f'XGBRegressor ({n_estimators_2}) {param_2}': xgb_study_2
}
result = evaluate_model(models, X_train_full, X_test_full, y_train, y_test)
result

,Train R2,CV R2,Test R2,Train RMSE,CV RMSE,Test RMSE,Train MAE,CV MAE,Test MAE,Model
1,0.9982,0.9245,0.9011,0.4668,3.0407,3.5902,0.3321,1.1536,1.1809,XGBRegressor (1778) {'objective': 'reg:squared...
0,0.9976,0.9218,0.8887,0.5474,3.0953,3.8085,0.3797,1.1987,1.2540,XGBRegressor (129) {'objective': 'reg:squarede...


In [ ]:
result.to_csv("tuning_imputed_all (XGB).csv", index=False)

### LightGBM

#### Study 1

In [ ]:
def objective(trial):

    dtrain = lgb.Dataset(X_train_full, label=y_train)

    params = {
        "objective": "regression",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "learning_rate": 0.1,
        "max_depth": trial.suggest_int("max_depth", 1, 30),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 1.0),
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1),
        "feature_fraction": trial.suggest_uniform("feature_fraction", 0.1, 1),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "rmse")

    cv = KFold(n_splits=5, shuffle=True, random_state=0)
    scores = lgb.cv(params, dtrain, 
                    num_boost_round=2000, 
                    early_stopping_rounds=100,
                    callbacks=[pruning_callback],
                    metrics='rmse', 
                    folds=cv)

    mean_score = scores['rmse-mean'][-1]
    return mean_score

pruner = optuna.pruners.MedianPruner(n_warmup_steps=10)
study = optuna.create_study(pruner=pruner, direction='minimize')
study.optimize(objective, n_trials=1000)


print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2020-10-29 21:17:40,056] A new study created in memory with name: no-name-5d65a332-390c-488b-9427-9bc932657243
[I 2020-10-29 21:17:44,008] Trial 0 finished with value: 3.6203947750562797 and parameters: {'max_depth': 14, 'num_leaves': 146, 'lambda_l1': 5.731625070646791e-08, 'lambda_l2': 7.333671390799928e-05, 'bagging_freq': 3, 'bagging_fraction': 0.2905620179492402, 'feature_fraction': 0.921925584999252, 'min_child_samples': 80}. Best is trial 0 with value: 3.6203947750562797.
[I 2020-10-29 21:17:47,255] Trial 1 finished with value: 3.737334338126395 and parameters: {'max_depth': 30, 'num_leaves': 68, 'lambda_l1': 1.7577204249101408e-06, 'lambda_l2': 5.273473576887913e-06, 'bagging_freq': 4, 'bagging_fraction': 0.1782267717603842, 'feature_fraction': 0.7397123377137729, 'min_child_samples': 62}. Best is trial 0 with value: 3.6203947750562797.
[I 2020-10-29 21:17:52,035] Trial 2 finished with value: 3.3422473648604667 and parameters: {'max_depth': 24, 'num_leaves': 166, 'lambda_l1'

Number of finished trials: 1000
Best trial:
  Value: 2.952685841368562
  Params: 
    max_depth: 9
    num_leaves: 201
    lambda_l1: 0.003523071594529814
    lambda_l2: 1.0102387966692847e-05
    bagging_freq: 2
    bagging_fraction: 0.9725933478532838
    feature_fraction: 0.5970824526937236
    min_child_samples: 5


In [ ]:
# Get best params then add to param_1
study_1_params = study.best_params
param_1 = {
    "objective": "regression",
    "verbosity": -1,
    "boosting_type": "gbdt",
    "learning_rate": 0.1,
}
param_1.update(study_1_params)
param_1

{'bagging_fraction': 0.9725933478532838,
 'bagging_freq': 2,
 'boosting_type': 'gbdt',
 'feature_fraction': 0.5970824526937236,
 'lambda_l1': 0.003523071594529814,
 'lambda_l2': 1.0102387966692847e-05,
 'learning_rate': 0.1,
 'max_depth': 9,
 'min_child_samples': 5,
 'num_leaves': 201,
 'objective': 'regression',
 'verbosity': -1}

In [ ]:
dtrain = lgb.Dataset(X_train_full, label=y_train)

cv = KFold(
    n_splits=5, 
    shuffle=True, 
    random_state=0
)
history = lgb.cv(
    param_1, dtrain, 
    num_boost_round=2000,
    early_stopping_rounds=100,
    metrics='rmse', 
    folds=cv
)
n_estimators_1 = pd.DataFrame(history).shape[0]
n_estimators_1

91

#### Study 2

In [ ]:
def objective(trial):

    dtrain = lgb.Dataset(X_train_full, label=y_train)

    params = {
        "objective": "regression",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "learning_rate": 0.01,
        "max_depth": trial.suggest_int("max_depth", 1, 30),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 1.0),
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1),
        "feature_fraction": trial.suggest_uniform("feature_fraction", 0.1, 1),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "rmse")

    cv = KFold(n_splits=5, shuffle=True, random_state=0)
    scores = lgb.cv(params, dtrain, 
                    num_boost_round=2000, 
                    early_stopping_rounds=100,
                    callbacks=[pruning_callback],
                    metrics='rmse', 
                    folds=cv)

    mean_score = scores['rmse-mean'][-1]
    return mean_score

pruner = optuna.pruners.MedianPruner(n_warmup_steps=10)
study = optuna.create_study(pruner=pruner, direction='minimize')
study.optimize(objective, n_trials=1000)


print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2020-10-29 21:32:55,734] A new study created in memory with name: no-name-71821531-5fd2-414f-ad40-0097dd0662cf
[I 2020-10-29 21:33:17,695] Trial 0 finished with value: 3.0881719984582263 and parameters: {'max_depth': 18, 'num_leaves': 90, 'lambda_l1': 0.7458183548271522, 'lambda_l2': 3.772575332814447e-07, 'bagging_freq': 5, 'bagging_fraction': 0.5899387019798016, 'feature_fraction': 0.8733163930746921, 'min_child_samples': 10}. Best is trial 0 with value: 3.0881719984582263.
[I 2020-10-29 21:33:26,207] Trial 1 finished with value: 3.5863188004573003 and parameters: {'max_depth': 15, 'num_leaves': 137, 'lambda_l1': 0.20726834082025244, 'lambda_l2': 0.0018343314248672753, 'bagging_freq': 2, 'bagging_fraction': 0.8375402962212013, 'feature_fraction': 0.14935892168635717, 'min_child_samples': 95}. Best is trial 0 with value: 3.0881719984582263.
[I 2020-10-29 21:33:38,678] Trial 2 finished with value: 3.33996920054807 and parameters: {'max_depth': 14, 'num_leaves': 46, 'lambda_l1': 2.31

Number of finished trials: 1000
Best trial:
  Value: 3.0132152619414745
  Params: 
    max_depth: 15
    num_leaves: 124
    lambda_l1: 7.558020915496281e-06
    lambda_l2: 0.06865296219872195
    bagging_freq: 4
    bagging_fraction: 0.6794166121180095
    feature_fraction: 0.940557615470706
    min_child_samples: 5


In [ ]:
# Get best params then add to param_2
study_2_params = study.best_params
param_2 = {
    "objective": "regression",
    "verbosity": -1,
    "boosting_type": "gbdt",
    "learning_rate": 0.1,
}
param_2.update(study_2_params)
param_2

{'bagging_fraction': 0.6794166121180095,
 'bagging_freq': 4,
 'boosting_type': 'gbdt',
 'feature_fraction': 0.940557615470706,
 'lambda_l1': 7.558020915496281e-06,
 'lambda_l2': 0.06865296219872195,
 'learning_rate': 0.1,
 'max_depth': 15,
 'min_child_samples': 5,
 'num_leaves': 124,
 'objective': 'regression',
 'verbosity': -1}

In [ ]:
dtrain = lgb.Dataset(X_train_full, label=y_train)

cv = KFold(
    n_splits=5, 
    shuffle=True, 
    random_state=0
)
history = lgb.cv(
    param_2, dtrain, 
    num_boost_round=2000,
    early_stopping_rounds=100,
    metrics='rmse', 
    folds=cv
)
n_estimators_2 = pd.DataFrame(history).shape[0]
n_estimators_2

58

#### Evaluation

In [ ]:
lgb_study_1 = LGBMRegressor(**param_1, n_estimators=n_estimators_1)
lgb_study_2 = LGBMRegressor(**param_2, n_estimators=n_estimators_2)

models = {
    f'LGBMRegressor ({n_estimators_1}) {param_1}': lgb_study_1,
    f'LGBMRegressor ({n_estimators_2}) {param_2}': lgb_study_2
}
result = evaluate_model(models, X_train, X_test, y_train, y_test)
result

,Train R2,CV R2,Test R2,Train RMSE,CV RMSE,Test RMSE,Train MAE,CV MAE,Test MAE,Model
0,0.9945,0.9247,0.8971,0.8181,3.0328,3.6614,0.4807,1.2189,1.2516,"LGBMRegressor (91) {'objective': 'regression',..."
1,0.9871,0.9178,0.8995,1.2570,3.1700,3.6183,0.5397,1.2557,1.2717,"LGBMRegressor (58) {'objective': 'regression',..."


#### Study 3

In [ ]:
def objective(trial):

    dtrain = lgb.Dataset(X_train_full, label=y_train)

    param = param_2
    param["learning_rate"] = trial.suggest_uniform('learning_rate', 0.001, 0.01)

    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "rmse")

    cv = KFold(n_splits=5, shuffle=True, random_state=0)
    scores = lgb.cv(params, dtrain, 
                    num_boost_round=2000, 
                    early_stopping_rounds=100,
                    callbacks=[pruning_callback],
                    metrics='rmse', 
                    folds=cv)

    mean_score = scores['rmse-mean'][-1]
    return mean_score

pruner = optuna.pruners.MedianPruner(n_warmup_steps=10)
study = optuna.create_study(pruner=pruner, direction='minimize')
study.optimize(objective, n_trials=1000)


print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2020-10-29 22:12:25,813] A new study created in memory with name: no-name-d376289a-673b-4065-994f-b1ead3853e5f
[W 2020-10-29 22:12:25,823] Trial 0 failed because of the following error: NameError("name 'params' is not defined",)
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/optuna/study.py", line 799, in _run_trial
    result = func(trial)
  File "<ipython-input-33-b628921eda11>", line 11, in objective
    scores = lgb.cv(params, dtrain,
NameError: name 'params' is not defined


NameError: ignored

In [ ]:
# Get best params then add to param_3
param_3 = param_2.copy()
param_3["learning_rate"] = study.best_params["learning_rate"]
param_3

In [ ]:
dtrain = lgb.Dataset(X_train_full, label=y_train)

cv = KFold(
    n_splits=5, 
    shuffle=True, 
    random_state=0
)
history = lgb.cv(
    param_3, dtrain, 
    num_boost_round=2000,
    early_stopping_rounds=100,
    metrics='rmse', 
    folds=cv
)
n_estimators_3 = pd.DataFrame(history).shape[0]
n_estimators_3

#### Evaluation

In [ ]:
lgb_study_1 = LGBMRegressor(**param_1, n_estimators=n_estimators_1)
lgb_study_2 = LGBMRegressor(**param_2, n_estimators=n_estimators_2)
lgb_study_3 = LGBMRegressor(**param_3, n_estimators=n_estimators_3)

models = {
    f'LGBMRegressor ({n_estimators_1}) {param_1}': lgb_study_1,
    f'LGBMRegressor ({n_estimators_2}) {param_2}': lgb_study_2,
    f'LGBMRegressor ({n_estimators_3}) {param_3}': lgb_study_3
}
result = evaluate_model(models, X_train_full, X_test_full, y_train, y_test)
result

In [ ]:
result.to_csv("tuning_dropna_all (LGB).csv", index=False)